In [1]:
rm(list = ls())
pcks <- c('stm', 'dplyr', 'quanteda', 'data.table', 'Hmisc', 'reticulate', 
          'purrr', 'svMisc', 'data.table', 'lubridate', 'tidyr', 'tictoc')
sapply(pcks, require, character.only = T, quietly = T)
require()

stm v1.3.5 successfully loaded. See ?stm for help. 
 Papers, resources, and other materials at structuraltopicmodel.com


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Package version: 2.0.1

Parallel computing: 2 of 8 threads used.

See https://quanteda.io for tutorials and examples.


Attaching package: ‘quanteda’


The following object is masked from ‘jupyter:irkernel’:

    View


The following object is masked from ‘package:utils’:

    View



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘lattice’


The following object is masked from ‘package:stm’:

    cloud



Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, 

stm      dplyr   quanteda data.table      Hmisc reticulate      purrr 
      TRUE       TRUE       TRUE       TRUE       TRUE       TRUE       TRUE 
    svMisc data.table  lubridate      tidyr     tictoc 
      TRUE       TRUE       TRUE       TRUE       TRUE

## 1. Read data

### 1.1. `Utilities.py`로부터 `read_pickle_file` method를 이용해 필요한 pickle 파일을 로드

In [2]:
source_python("Utilities.py")
국회.19대.doc <- read_pickle_file("./result/국회_19대_BOW.pickle")
국회.19대.vocab <- read_pickle_file("./result/국회_19대_dict.pickle")
국회.19대.meta <- read_pickle_file("./result/국회_19대_토큰.pkl")

국회.19대.meta[2, ]

,회의록ID,대수,위원회구분,위원회,회차,회의일자,회의장소,발언ID,발언순서,안건,발언자,발언내용,token
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<list>,<chr>,<chr>,<list>
1,41725,19,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814716,2,FRONT,김기현 위원,반갑습니다. 앉아서 인사드리겠습니다. 새누리당 김기현 위원입니다. 앞으로 우리 위원님들 잘 모시고 우리 위원회가 국회 내에서 가장 모범적인 위원회로 운영될 수 있도록 잘 뒷바라지하도록 하겠습니다. 만나 뵙게 되서 대단히 반갑습니다.,"인사 , 누리 , 당 , 김기현 , 위원 , 앞 , 위원 , 위원회 , 국회 , 모범 , 위원회 , 운영 , 뒷바라지"


In [9]:
국회.19대.doc <- 국회.19대.doc.o

### 1.2. token 열을 `stm`에 맞게 list of matrices로 parse

In [10]:
parse_to_m = function(l){
    rbind(l[[1]]+1, l[[2]]) # Python과 R의 indexing 차이
}

# for (i in 1:length(국회.19대.doc)){
for (i in 1){
    progress(i, length(국회.19대.doc), progress.bar = T)
    doci = 국회.19대.doc[[i]]
    doci = lapply(doci, parse_to_m)
    mat = data.frame(doci) %>% as.matrix
    colnames(mat) = NULL
    국회.19대.doc[[i]] = mat
}

# 국회.19대.meta %>% select(위원회, 회차, 회의일자, 발언자) %>% describe()
국회.19대.doc

#### 1.2.1 Save or load RDS

In [6]:
out <- prepDocuments(국회.19대.doc, 국회.19대.vocab, 국회.19대.meta, lower.thresh = 0)
saveRDS(out, './result/processed.data.for.stm.rds')